# Corridor Detection Diagnostics

In [ ]:

from pathlib import Path
import sys

REPO_ROOT = Path.cwd().resolve()
SRC_PATH = REPO_ROOT / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

print(f"Using src path: {SRC_PATH}")


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

from behavioral_analysis.io.json_parser import parse_json_file
from behavioral_analysis.io.dataframe_builder import extract_events_by_type
from behavioral_analysis.processing import (
    compute_corridor_artifacts,
)

plt.style.use('seaborn-v0_8')

from IPython.display import display

In [ ]:

JSON_PATH = Path('/groups/spruston/home/moharb/DELTA_Behavior/Log BM35 2025-09-22 session 1.json')

if not JSON_PATH.exists():
    raise FileNotFoundError(f"JSON log not found: {JSON_PATH}")

print(f'Loading events from: {JSON_PATH.name}')

raw_events = parse_json_file(JSON_PATH, verbose=False)
frames = extract_events_by_type(raw_events, verbose=False)

print(f"Total events: {len(raw_events):,}")
print(f"Available event types: {sorted(frames.keys())}")


In [ ]:

artifacts = compute_corridor_artifacts(
    cue_state_df=frames.get('Cue State'),
    position_df=frames.get('Path Position'),
    cue_result_df=frames.get('Cue Result'),
    verbose=False,
)

corridor_info = artifacts.corridor_info
cue_matches = artifacts.cue_matches
position_loops = artifacts.position_loops

print(f"Corridors detected: {len(corridor_info)}")
print(f"Position loops: {len(position_loops)}")
print(f"Matched cues: {0 if cue_matches is None else len(cue_matches)}")

corridor_info.head()


In [ ]:

if cue_matches is not None:
    cue_matches.head()
else:
    print('Cue match table is empty.')


In [ ]:

path_position_df = frames.get('Path Position')
cue_results_df = artifacts.cue_result_with_corridors

if path_position_df is not None and not corridor_info.empty and cue_results_df is not None:
    corridor_id = 0
    loop = corridor_info[corridor_info['corridor_id'] == corridor_id].iloc[0]

    mask = (path_position_df['time'] >= loop['start_time']) & (path_position_df['time'] <= loop['end_time'])
    subset = path_position_df.loc[mask, ['time', 'position']].copy()
    subset['time_seconds'] = subset['time'] / 1000.0

    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(subset['time_seconds'], subset['position'], color='steelblue', linewidth=1.5)

    hits = cue_results_df[cue_results_df['corridor_id'] == corridor_id]
    if not hits.empty:
        ax.vlines(hits['time'] / 1000.0, ymin=subset['position'].min(), ymax=subset['position'].max(),
                  colors='tomato', linestyles='--', alpha=0.6, label='Cue hits')

    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Path position (a.u.)')
    ax.set_title(f'Corridor {corridor_id}: path trajectory with cue hits')
    ax.legend()
    plt.tight_layout()
    plt.show()
else:
    print('Not enough data to visualize the first corridor.')


In [ ]:

summary_cols = [
    'corridor_id',
    'start_time',
    'end_time',
    'num_cue_results',
    'num_matched_cues',
    'loop_complete',
]

if not corridor_info.empty:
    display(corridor_info[summary_cols].head(12))
else:
    print('Corridor summary unavailable.')
